# OpenAI Q&A
ref: https://platform.openai.com/docs/tutorials/web-qa-embeddings
https://github.com/openai/openai-cookbook/blob/main/apps/web-crawl-q-and-a/web-qa.ipynb

In [2]:
import pandas as pd
outputpath="./output/text/processed"
df = pd.read_csv(outputpath+'/split.csv', index_col=0)
df.head()

,text,n_tokens
0,u cmpe faculty faculty emeriti ron mak.php. ...,160
1,u cmpe faculty tenure line ahmet bindal.php. ...,161
2,.,1
3,u cmpe research research areas. Research A...,154
4,u cmpe academic cybersecurity program cybersec...,165


In [4]:
#enter open ai key in a popup window
import os
import getpass

os.environ['OPENAI_API_KEY'] = getpass.getpass('OpenAI API Key:')

In [7]:
import openai

df['embeddings'] = df.text.apply(lambda x: openai.Embedding.create(input=x, engine='text-embedding-ada-002')['data'][0]['embedding'])

AuthenticationError: No API key provided. You can set your API key in code using 'openai.api_key = <API-KEY>', or you can set the environment variable OPENAI_API_KEY=<API-KEY>). If your API key is stored in a file, you can point the openai module at it with 'openai.api_key_path = <PATH>'. You can generate API keys in the OpenAI web interface. See https://onboard.openai.com for details, or email support@openai.com if you have any questions.

In [ ]:
df.to_csv(outputpath+'/embeddings.csv')
df.head()